In [1]:
import gc

import pandas as pd

In [2]:
# it's okay for 16GB memory
CHUNK_SIZE = 10000000

In [3]:
header = True
count = 0
for df in pd.read_csv("in_rolls_state_name_age_sex.csv.gz", chunksize=CHUNK_SIZE):
    # Take out null elector_name and change to upper case
    df.dropna(subset=["elector_name"], inplace=True)
    df.elector_name = df.elector_name.str.lower()
    # Take out null and invalid age
    df.dropna(subset=["age"], inplace=True)
    # df.drop(df[df.age.str.len() > 3].index, inplace=True)
    # df.age = df.age.astype(int)
    df.age = pd.to_numeric(df.age, errors="coerce")
    df.drop(df[df.age > 130].index, inplace=True)
    df["birth_year"] = 2017 - df.age
    # df[df.age > 100].count()
    # Take out null in sex
    df.dropna(subset=["sex"], inplace=True)
    # Change lower case
    df.sex = df.sex.str.lower()
    # when we have more than 2 words, we should use the 1st word as first name
    # we can ignore rows where first_name < 2 characters
    # same for where last_name < 2 characters
    # and we can write than when we are describing the results
    # and i suppose we need to either ignore names with just 1 word though we can assume 1 word names (not 1 character) as the first name
    df["name_wc"] = df.elector_name.apply(lambda c: len(c.split()))
    df["first_name"] = None
    df["last_name"] = None
    df.loc[df.name_wc == 1, "first_name"] = df.elector_name
    df.loc[df.name_wc >= 2, "first_name"] = df.elector_name.apply(
        lambda c: c.split()[0]
    )
    df.loc[df.name_wc >= 2, "last_name"] = df.elector_name.apply(
        lambda c: c.split()[-1]
    )

    df.loc[df.first_name.notnull() & (df.first_name.str.len() <= 2), "first_name"] = (
        None
    )
    df.loc[df.last_name.notnull() & (df.last_name.str.len() <= 2), "last_name"] = None
    del df["name_wc"]

    # df[df.first_name.isnull()]
    # df[df.last_name.isnull()]

    df.to_csv(
        "in_rolls_state_name_age_sex_fn_ln.csv.gz",
        mode="a",
        header=header,
        index=False,
        compression="gzip",
    )
    header = False
    del df
    gc.collect()
    count += 1
    print(count)
    # break

/home/t-9adar/.conda/envs/env38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60


In [ ]:
# Resource List: procs=2,mem=16gb
# Resources Used: cput=02:09:36,vmem=6487812kb,walltime=02:09:45,mem=13009344kb,energy_used=0